In [15]:
import requests
import logging
import csv
from google.colab import files
from IPython.display import display

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def fetch_cve_data():
    """
    Fetches CVE data from the NVD API.
    Returns a JSON response containing vulnerabilities.
    """
    url = "https://services.nvd.nist.gov/rest/json/cves/2.0?startIndex=0&resultsPerPage=10"
    logging.info(f"Fetching data from: {url}")

    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed: {e}")
        return None

def save_cve_data(cve_data, severity_level):
    """
    Saves CVE data filtered by severity level to a CSV file.
    """
    if cve_data is None:
        logging.error("Error: cve_data is not available.")
        return

    severity_levels = {"low": 1, "medium": 2, "high": 3, "critical": 4}
    user_severity = severity_levels.get(severity_level.lower(), None)

    if user_severity is None:
        logging.error("Invalid severity level provided. Please choose from low, medium, high, or critical.")
        return

    logging.info("Processing CVE Data")
    file_name = "cve_data.csv"

    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["CVE ID", "Publish Date", "Last Modified", "Source", "Description", "Base Severity", "Attack Vector"])

        for item in cve_data.get("vulnerabilities", []):
            cve_info = item.get("cve", {})
            cve_id = cve_info.get("id", "N/A")
            publish_date = cve_info.get("published", "N/A")
            last_modified = cve_info.get("lastModified", "N/A")
            source = cve_info.get("sourceIdentifier", "N/A")
            description = cve_info.get("descriptions", [{}])[0].get("value", "N/A")

            impact = cve_info.get("metrics", {}).get("cvssMetricV31", [{}])[0].get("cvssData", {})
            base_severity = impact.get("baseSeverity", "N/A")
            attack_vector = impact.get("attackVector", "N/A")

            if base_severity.lower() in severity_levels and severity_levels[base_severity.lower()] >= user_severity:
                logging.warning(f"High severity CVE detected - {cve_id} ({base_severity})")
                writer.writerow([cve_id, publish_date, last_modified, source, description, base_severity, attack_vector])

    logging.info(f"CVE data saved to {file_name}")
    files.download(file_name)

def display_cve_data(cve_data, severity_level):
    """
    Displays CVE data filtered by severity level in a structured format.
    """
    if cve_data is None:
        logging.error("Error: cve_data is not available.")
        return

    severity_levels = {"low": 1, "medium": 2, "high": 3, "critical": 4}
    user_severity = severity_levels.get(severity_level.lower(), None)

    if user_severity is None:
        logging.error("Invalid severity level provided. Please choose from low, medium, high, or critical.")
        return

    logging.info("Processing CVE Data")
    filtered_data = []

    for item in cve_data.get("vulnerabilities", []):
        cve_info = item.get("cve", {})
        cve_id = cve_info.get("id", "N/A")
        publish_date = cve_info.get("published", "N/A")
        last_modified = cve_info.get("lastModified", "N/A")
        source = cve_info.get("sourceIdentifier", "N/A")
        description = cve_info.get("descriptions", [{}])[0].get("value", "N/A")

        impact = cve_info.get("metrics", {}).get("cvssMetricV31", [{}])[0].get("cvssData", {})
        base_severity = impact.get("baseSeverity", "N/A")
        attack_vector = impact.get("attackVector", "N/A")

        if base_severity.lower() in severity_levels and severity_levels[base_severity.lower()] >= user_severity:
            filtered_data.append([cve_id, publish_date, last_modified, source, description, base_severity, attack_vector])

    display(filtered_data)  # Display structured output

# Set default severity level (Modify as needed)
severity_level = "high"

# Fetch and process CVE data
detected_cve_data = fetch_cve_data()
display_cve_data(detected_cve_data, severity_level)
save_cve_data(detected_cve_data, severity_level)




[['CVE-1999-0084',
  '1990-05-01T04:00:00.000',
  '2024-11-20T23:27:48.653',
  'cve@mitre.org',
  'Certain NFS servers allow users to use mknod to gain privileges by creating a writable kmem device and setting the UID to 0.',
  'HIGH',
  'LOCAL']]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>